In [ ]:
from datasets import load_dataset

In [ ]:
data_files = {"train": "all-nli-qwen-reranker-train.parquet",
              "test": "all-nli-qwen-reranker-test.parquet",
              "dev": "all-nli-qwen-reranker-dev.parquet" }
train_dataset = load_dataset("parquet", data_files=data_files, split="train")

In [ ]:
train_dataset

In [ ]:
train_dataset[0]

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model_name = "answerdotai/ModernBERT-base"
model = SentenceTransformer(model_name, device="cuda")

In [ ]:
model.to('cuda')

In [ ]:
from sentence_transformers.losses import CoSENTLoss
loss = CoSENTLoss(model)

In [ ]:
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/ModernSBERT",
    # Optional training parameters:
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=False,
    bf16=True,
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
)

In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction

# Load the STSB dataset (https://huggingface.co/datasets/sentence-transformers/stsb)
eval_dataset = load_dataset("parquet", data_files=data_files, split="dev")

# Initialize the evaluator
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=eval_dataset["sentence1"],
    sentences2=eval_dataset["sentence2"],
    scores=eval_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name="all-nli-dev",
)

In [ ]:
test_dataset = load_dataset("parquet", data_files=data_files, split="test")

In [ ]:
test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=test_dataset["sentence1"],
    sentences2=test_dataset["sentence2"],
    scores=test_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
)

In [ ]:
len(test_dataset)

In [ ]:
test_evaluator(model)

In [ ]:
from sentence_transformers import SentenceTransformerTrainer
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
    evaluator=dev_evaluator,
)
trainer.train()

In [ ]:
model.save_pretrained("ModernSBERT-base-all-nli-qwen-reranker/final")

In [ ]:
model = SentenceTransformer("./ModernSBERT-base-all-nli-qwen-reranker/final")

In [ ]:
test_evaluator(model)